# MSiA 423 - Cloud Engineering for Data Science - Final Project
## Group 7: Alejandra Lelo de Larrea Ibarra, Bannasorn Paspanthong, Ruben Nakano, Samuel Swain
# Clean data

In [1]:
# Libraries 
import numpy as np
import pandas as pd
import zipfile
import time
import re
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

In [2]:
# Read raw data
#df_raw = pd.read_csv("../02_Data/raw_data.csv")

In [3]:
# --- Read csv file directly from the zip ---
archive = zipfile.ZipFile('../02_Data/archive.zip', 'r')
files = archive.namelist()

# --- Load data sets --- 
# Business class 
with archive.open(files[1]) as csvfile:   
    df_business = pd.read_csv(csvfile)
    df_business["class"] = "Business"
    print("Buisness data shape: " + str(df_business.shape))
    
# Economy class 
with archive.open(files[2]) as csvfile:   
    df_economy = pd.read_csv(csvfile)
    df_economy["class"] = "Economy"
    print("Economy data shape: " + str(df_economy.shape))
    
# --- Bind business and economy class --- 
df_raw = pd.concat([df_business,df_economy], axis = 0)
print("Raw data shape: " + str(df_raw.shape))

Buisness data shape: (93487, 12)
Economy data shape: (206774, 12)
Raw data shape: (300261, 12)


In [4]:
# Find null values 
df_raw.isna().sum()

date          0
airline       0
ch_code       0
num_code      0
dep_time      0
from          0
time_taken    0
stop          0
arr_time      0
to            0
price         0
class         0
dtype: int64

In [5]:
# Header 
df_raw.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,class
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612",Business
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612",Business
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,Mumbai,"42,220",Business
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"44,450",Business
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"46,690",Business


In [6]:
# --- Cleaning aux functions --- 
def get_duration(text):
    pattern = re.match(r'[0-9]+\.[0-9]{1,2}h m', text)
    if(pattern != None):
        # Extract hours
        hours = re.search(r'^[0-9]+\.', text)
        hours = int(hours.group().replace('.',''))
        
        # Extract minutes
        minutes = re.search(r'\.[0-9]{1,2}h', text)
        minutes = float(minutes.group().replace('h', ''))*60
        minutes = int(round(minutes))
    else:
        # Extract hours
        hours = re.search(r'^[0-9]+h', text)
        hours = int(hours.group().replace('h',''))
        # Extract minutes
        minutes = re.search(r'[0-5][0-9]m', text)
        minutes = int(minutes.group().replace('m', ''))
    
    # Calculate duration in hours
    duration = round(hours + minutes/60,2)
    
    return duration

def get_stops(text):
    # Regex for number of stops 
    pattern = re.search(r'(non-stop|1-stop|2\+-stop)', text)
    stops = pattern.group()
    
    # Convert to numeric output
    if(stops == 'non-stop'):
        return("0")
    elif(stops == '1-stop'):
        return("1")
    else:
        return("2")
    
def bucket_hours(time):
    # Regex for hour of the day
    pattern = re.search(r'^[0-2][0-9]\:', time)
    hour = int(pattern.group().replace(":",""))
    
    # Early morning 04:00-7:59
    if(hour>=4 and hour<8): 
        return("Early_Morning")
    # Morning 8:00-11:59
    elif(hour>=8 and hour<12):
        return("Morning")
    # Afternoon 12:00 - 15:59
    elif(hour>=12 and hour<16):
        return("Afternoon")
    # Evening 16:00 19:59
    elif(hour>=16 and hour<20):
        return("Evening")
    # Night: 20:00-23:59
    elif(hour>=20 and hour<24):
        return("Night")
    # Late night: 00:00-03:59
    else:
        return("Late_Night")

In [7]:
# --- Clean data --- 
df_clean = df_raw.copy()

# Change column date to book_date
df_clean.columns.values[0] = "book_date"
   
# Get flight code
df_clean["flight"] = df_raw["ch_code"] + "-" + df_raw["num_code"].astype(str)

# Bucket dep_time 
df_clean["departure_time"] = df_raw.dep_time.apply(lambda x: bucket_hours(x))

# Change column "from" to "origin"
df_clean.columns.values[5] = "origin"

# Convert time taken in durations as hours between departure and arrival
df_clean["duration"] = df_raw.time_taken.apply(lambda x: get_duration(x))

# Get number of stops 
df_clean["stops"] = df_raw["stop"].apply(lambda x: get_stops(x))

# Bucket arr_time 
df_clean["arrival_time"] = df_raw.arr_time.apply(lambda x: bucket_hours(x))

# Change column "to" to "destination"
df_clean.columns.values[9] = "destination"

# Convert price to numeric 
df_clean["price"] = df_raw["price"].str.replace(",", "").astype(int)

# Select features
df_clean = df_clean[["book_date", "airline", "flight", "class", "departure_time",
                    "origin", "duration", "stops", "arrival_time", "destination", "price"]]

# Show clean dataset 
df_clean.head()

,book_date,airline,flight,class,departure_time,origin,duration,stops,arrival_time,destination,price
0,11-02-2022,Air India,AI-868,Business,Evening,Delhi,2.00,0,Night,Mumbai,25612
1,11-02-2022,Air India,AI-624,Business,Evening,Delhi,2.25,0,Night,Mumbai,25612
2,11-02-2022,Air India,AI-531,Business,Night,Delhi,24.75,1,Night,Mumbai,42220
3,11-02-2022,Air India,AI-839,Business,Night,Delhi,26.50,1,Night,Mumbai,44450
4,11-02-2022,Air India,AI-544,Business,Evening,Delhi,6.67,1,Night,Mumbai,46690


In [8]:
# --- Save file into a CSV --- 
df_clean.to_csv("../02_Data/clean_data.csv", index = False)